In [139]:
import pandas as pd
import numpy as np
import re

In [140]:
df = pd.read_csv('../../data/raw/realestates_kh_v2.csv')

In [141]:
df = df[df['address_subdivision'] == 'Phnom Penh']

In [142]:
df['land_area'] = df['land_area'].replace(0.0, np.nan).replace('', np.nan)

In [143]:
# def extract_land_area(text):
#     if pd.isna(text):
#         return None
    
#     # Step 1: Direct land area extraction (e.g., "100 sqm")
#     direct_pattern = r'(\d+\.?\d*)\s*(?:sqm|m²|m2|square\s*meters?)\b'
#     direct_match = re.search(direct_pattern, text, re.IGNORECASE)
#     if direct_match:
#         return float(direct_match.group(1))
    
#     # Step 2: Extract land dimensions (e.g., "4m x 20m") with contextual keywords
#     dim_pattern = r'(\d+\.?\d*)\s*(?:m\s*)?[x*]\s*(\d+\.?\d*)\s*(?:m\b)'
#     dim_matches = list(re.finditer(dim_pattern, text, re.IGNORECASE))
    
#     for match in dim_matches:
#         start, end = match.span()
#         window = text[max(0, start-30):min(len(text), end+30)].lower()
#         # Check for contextual keywords near the dimensions
#         if re.search(r'\b(?:land|plot|ground|property|size)\b', window):
#             length = float(match.group(1))
#             width = float(match.group(2))
#             return length * width
    
#     # Step 3: Fallback to the first dimension pattern found (if any)
#     if dim_matches:
#         length = float(dim_matches[0].group(1))
#         width = float(dim_matches[0].group(2))
#         return length * width
    
#     return None

In [144]:
def extract_land_area(text):
    if pd.isna(text):
        return None
    def parse_float_number(num_str):
        if num_str is None:
            return None
        return float(num_str.replace(',', '.'))
    number_pattern = r'\d+(?:[.,]\d*)?'

    # Step 1: Direct land area extraction (e.g., "100 sqm", "9,2 m2")
    direct_pattern = rf'({number_pattern})\s*(?:sqm|m²|m2|square\s*meters?)\b'
    direct_match = re.search(direct_pattern, text, re.IGNORECASE)
    if direct_match:
        extracted_area = parse_float_number(direct_match.group(1))
        # Apply the condition: if extracted_area < 30, return None
        if extracted_area is not None and extracted_area < 30:
            return None
        return extracted_area
    
    # Step 2: Extract land dimensions (e.g., "4m x 20m", "9,5m*10m") with contextual keywords
    dim_pattern = rf'({number_pattern})\s*(?:m\s*)?[x*]\s*({number_pattern})\s*(?:m\b)'
    dim_matches = list(re.finditer(dim_pattern, text, re.IGNORECASE))
    
    for match in dim_matches:
        start, end = match.span()
        window = text[max(0, start-30):min(len(text), end+30)].lower()
        
        if re.search(r'\b(?:land|plot|ground|property|size)\b', window):
            length = parse_float_number(match.group(1))
            width = parse_float_number(match.group(2))
            
            if length is not None and width is not None:
                extracted_area = length * width
                # Apply the condition: if extracted_area < 30, return None
                if extracted_area < 30:
                    return None
                return extracted_area
    
    # Step 3: Fallback to the first dimension pattern found (if any)
    if dim_matches:
        length = parse_float_number(dim_matches[0].group(1))
        width = parse_float_number(dim_matches[0].group(2))
        
        if length is not None and width is not None:
            extracted_area = length * width
            # Apply the condition: if extracted_area < 30, return None
            if extracted_area < 30:
                return None
            return extracted_area
    
    return None

In [145]:
rows_to_reprocess = df['land_area'].apply(
    lambda x: pd.isna(x) or (pd.api.types.is_numeric_dtype(x) and (x == 0 or x < 30))
)

# Iterate only over the rows that meet the condition
for idx, row in df[rows_to_reprocess].iterrows():
    extracted_area = extract_land_area(row['information'])
    
    if extracted_area is not None:
        df.at[idx, 'land_area'] = extracted_area
    else:
        df.at[idx, 'land_area'] = np.nan

In [ ]:
df.loc[df['land_area'].apply(lambda x: pd.api.types.is_numeric_dtype(x) and x < 30), 'land_area'] = np.nana

In [167]:
c =0
for i, row in df.iterrows():
    
    if row['land_area'] < 30:
        row['land_area'] = np.nan

In [168]:
c =0
for i, row in df.iterrows():
    
    if row['land_area'] < 30:
        c += 1
print(c)

9


In [146]:
df.drop(columns='garages', inplace=True)

In [147]:
indexPrice = df[(df['price_display'] == 'POA')].index
df.drop(indexPrice, inplace=True)

In [148]:
# indexPrice = df[(df['price_display']<df['price_display'])].index
# # df.drop(indexPrice, inplace=True)

In [149]:
print(indexPrice)

Index([  16,  177,  211,  273,  340,  592, 1018, 1019, 1039, 1044, 1048, 1049,
       1065, 1078, 1079, 1103, 1113, 1122, 1135, 1137, 1179, 1198, 1243, 1256,
       1275, 1283, 1284, 1285, 1286, 1291, 1298, 1325, 1334, 1335, 1360, 1361,
       1463, 1481, 1482, 1483, 1484, 1485, 1486, 1490, 1509, 1514, 1525, 1532,
       1536, 1537, 1538, 1540, 1543, 1616, 1617, 1619, 1623, 1630, 1646, 1930,
       1940, 2132, 2181, 2184, 2189, 2244, 2291, 2296, 2720, 2795, 2797, 2798,
       2799, 2800, 2801, 2802, 2843, 3023, 3103, 3110, 3461, 3509, 3520, 3524,
       3545, 3572, 3695, 3743, 3894, 3907, 3976, 3980, 4090, 4114, 4149, 4171,
       4196],
      dtype='int64')


In [150]:
df.isnull().sum()

Unnamed: 0                0
id                        0
headline                  0
price_display            13
rent_display           3514
bedrooms                941
bathrooms               948
land_area               604
thumbnail_url             0
thumbnail_urls            0
location                  0
address_subdivision       0
address_locality          0
address_line_2            9
address_line_1          148
category_name             0
is_parent                 0
type                      0
information              38
source_url                0
language                  0
latitude                  0
longitude                 0
dtype: int64

In [159]:
df.to_csv('../../data/processed/realestates_kh_v2_3.csv', index=False)